In [ ]:
from transformers import AutoTokenizer, MarianMTModel

src = "en"  # source language
trg = "mr"  # target language

model_name = f"Helsinki-NLP/opus-mt-{src}-{trg}"
model = MarianMTModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

sample_text = "where is hospital nearby?"
batch = tokenizer([sample_text], return_tensors="pt")

generated_ids = model.generate(**batch)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load the model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

source_texts = [
    "This is a pen.", "How are you?", "Where is the book?", "She is going to school.", 
    "The cat is on the roof.", "He is a doctor.", "I love to read books.", "We are learning to code.", 
    "They are playing football.", "What is your name?", "The weather is nice today.", 
    "It is raining heavily.", "I am very tired.", "You should eat healthy food.", 
    "She likes to sing.", "The train is late.", "I have a meeting at 10 AM.", 
    "Can you help me with this?", "The phone is ringing.", "We need to go now.", 
    "The sun is shining brightly.", "This is my favorite movie.", "He is very talented.", 
    "We will visit the museum tomorrow.", "The food tastes delicious.", "She is a good dancer.", 
    "He is waiting for the bus.", "What time is it?", "This is a difficult question.", 
    "The flowers are blooming.", "I need to finish my homework.", "Do you know how to cook?", 
    "The dog is barking loudly.", "She is reading a novel.", "The kids are playing outside.", 
    "What did you say?", "Please pass the salt.", "The store is closed.", "He is very kind.", 
    "This is the last one.", "I need a new phone.", "The movie starts at 7 PM.", 
    "She is wearing a red dress.", "I will call you later.", "We are planning a trip.", 
    "They are coming home soon.", "This is a beautiful painting.", "The concert was amazing.", 
    "I am learning French.", "The sun sets in the west.", "Can you hear me?", "It is very cold today.", 
    "He is writing a letter.", "The children are asleep.", "What is your favorite color?", 
    "She is a brilliant student.", "I forgot my keys.", "Do you have any questions?", 
    "The teacher is explaining the lesson.", "I will meet you at the park.", "Please be quiet.", 
    "The stars are shining in the sky.", "I have completed my project.", "The baby is crying.", 
    "She is watching a movie.", "I am making dinner.", "He is playing the guitar.", 
    "The train is arriving at the station.", "The house is very big.", "I am feeling better now.", 
    "The book is very interesting.", "She is going to the market.", "He is wearing a hat.", 
    "The birds are singing.", "The car is parked outside.", "I have lost my wallet.", 
    "She is baking a cake.", "He is studying for exams.", "The tree is very tall.", 
    "What is the problem?", "We need to buy groceries.", "He is painting the wall.", 
    "This is a serious issue.", "Can you open the door?", "The room is very clean.", 
    "I am going to the gym.", "The pizza is ready.", "He is a famous actor.", 
    "She is looking for her glasses.", "The hotel is fully booked.", "I will send you an email.", 
    "The water is boiling.", "We are visiting our grandparents.", "The road is under construction.", 
    "I am reading a magazine.", "The sky is clear.", "She is driving the car.", 
    "The dog is chasing the ball.", "The cake tastes sweet.", "He is playing a video game.", 
    "We are having a party tonight.", "The plane is taking off.", "It is very hot outside.", 
    "The movie was very entertaining.", "She is talking on the phone.", "I am learning how to swim."
]

reference_sentences = [
    "यह एक पेन है।", "आप कैसे हैं?", "किताब कहाँ है?", "वह स्कूल जा रही है।", 
    "बिल्ली छत पर है।", "वह एक डॉक्टर है।", "मुझे किताबें पढ़ना पसंद है।", "हम कोडिंग सीख रहे हैं।", 
    "वे फुटबॉल खेल रहे हैं।", "तुम्हारा नाम क्या है?", "आज मौसम अच्छा है।", 
    "तेज बारिश हो रही है।", "मैं बहुत थका हुआ हूँ।", "आपको स्वस्थ भोजन खाना चाहिए।", 
    "उसे गाना पसंद है।", "ट्रेन देर से है।", "मेरी 10 बजे बैठक है।", 
    "क्या आप इसमें मेरी मदद कर सकते हैं?", "फोन बज रहा है।", "हमें अब जाना चाहिए।", 
    "सूरज तेज़ चमक रहा है।", "यह मेरी पसंदीदा फिल्म है।", "वह बहुत प्रतिभाशाली है।", 
    "हम कल संग्रहालय जाएंगे।", "खाना बहुत स्वादिष्ट है।", "वह एक अच्छी नर्तकी है।", 
    "वह बस की प्रतीक्षा कर रहा है।", "समय क्या हो रहा है?", "यह एक कठिन सवाल है।", 
    "फूल खिल रहे हैं।", "मुझे अपना होमवर्क खत्म करना है।", "क्या आप खाना बनाना जानते हैं?", 
    "कुत्ता जोर से भौंक रहा है।", "वह एक उपन्यास पढ़ रही है।", "बच्चे बाहर खेल रहे हैं।", 
    "आपने क्या कहा?", "कृपया नमक पास करें।", "दुकान बंद है।", "वह बहुत दयालु है।", 
    "यह आखिरी है।", "मुझे एक नया फोन चाहिए।", "फिल्म 7 बजे शुरू होती है।", 
    "वह लाल पोशाक पहन रही है।", "मैं आपको बाद में कॉल करूंगा।", "हम एक यात्रा की योजना बना रहे हैं।", 
    "वे जल्द ही घर आ रहे हैं।", "यह एक सुंदर पेंटिंग है।", "संगीत कार्यक्रम अद्भुत था।", 
    "मैं फ्रेंच सीख रहा हूँ।", "सूरज पश्चिम में डूबता है।", "क्या आप मुझे सुन सकते हैं?", 
    "आज बहुत ठंड है।", "वह एक पत्र लिख रहा है।", "बच्चे सो रहे हैं।", "आपका पसंदीदा रंग कौन सा है?", 
    "वह एक प्रतिभाशाली छात्रा है।", "मैं अपनी चाबियाँ भूल गया।", "क्या आपके पास कोई सवाल है?", 
    "शिक्षक पाठ समझा रहा है।", "मैं पार्क में मिलूंगा।", "कृपया चुप रहें।", 
    "आसमान में सितारे चमक रहे हैं।", "मैंने अपना प्रोजेक्ट पूरा कर लिया है।", "बच्चा रो रहा है।", 
    "वह एक फिल्म देख रही है।", "मैं रात का खाना बना रहा हूँ।", "वह गिटार बजा रहा है।", 
    "ट्रेन स्टेशन पर पहुँच रही है।", "घर बहुत बड़ा है।", "अब मैं बेहतर महसूस कर रहा हूँ।", 
    "यह किताब बहुत दिलचस्प है।", "वह बाजार जा रही है।", "वह टोपी पहन रहा है।", 
    "पक्षी गा रहे हैं।", "गाड़ी बाहर खड़ी है।", "मेरा बटुआ खो गया है।", 
    "वह केक बना रही है।", "वह परीक्षाओं की तैयारी कर रहा है।", "पेड़ बहुत ऊँचा है।", 
    "समस्या क्या है?", "हमें किराने का सामान खरीदना है।", "वह दीवार पर रंग कर रहा है।", 
    "यह एक गंभीर मुद्दा है।", "क्या आप दरवाजा खोल सकते हैं?", "कमरा बहुत साफ है।", 
    "मैं जिम जा रहा हूँ।", "पिज्जा तैयार है।", "वह एक प्रसिद्ध अभिनेता है।", 
    "वह अपने चश्मे की तलाश कर रही है।", "होटल पूरी तरह से बुक है।", "मैं आपको एक ईमेल भेजूंगा।", 
    "पानी उबल रहा है।", "हम अपने दादा-दादी से मिलने जा रहे हैं।", "सड़क निर्माणाधीन है।", 
    "मैं एक पत्रिका पढ़ रहा हूँ।", "आसमान साफ है।", "वह कार चला रही है।", 
    "कुत्ता गेंद का पीछा कर रहा है।", "केक मीठा लगता है।", "वह वीडियो गेम खेल रहा है।", 
    "हम आज रात एक पार्टी कर रहे हैं।", "विमान उड़ान भर रहा है।", "बाहर बहुत गर्मी है।", 
    "फिल्म बहुत मनोरंजक थी।", "वह फोन पर बात कर रही है।", "मैं तैरना सीख रहा हूँ।"
]


# Sample input text

inputs = tokenizer(source_texts, return_tensors="pt", padding=True, truncation=True)

# Generate translations
translated = model.generate(**inputs)
predicted_sentences = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Reference translations (ground truth)


# Token-level evaluation
for ref_sent, pred_sent in zip(reference_sentences, predicted_sentences):
    ref_tokens = ref_sent.split()
    pred_tokens = pred_sent.split()

    # Truncate to the shortest sentence
    min_len = min(len(ref_tokens), len(pred_tokens))
    ref_tokens = ref_tokens[:min_len]
    pred_tokens = pred_tokens[:min_len]

    # Accuracy, Precision, Recall, F1-score
    print(f"Accuracy: {accuracy_score(ref_tokens, pred_tokens)}")
    print(classification_report(ref_tokens, pred_tokens))
    print(f"Confusion Matrix:\n{confusion_matrix(ref_tokens, pred_tokens)}")


In [4]:
from transformers import MarianMTModel, MarianTokenizer
from sklearn.metrics import classification_report, accuracy_score
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

# Load the model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# List of source texts and reference translations
source_texts = [
    "This is a pen.", "How are you?", "Where is the book?", "She is going to school.", 
    "The cat is on the roof.", "He is a doctor.", "I love to read books.", "We are learning to code.", 
    "They are playing football.", "What is your name?", "The weather is nice today.", 
    "It is raining heavily.", "I am very tired.", "You should eat healthy food.", 
    "She likes to sing.", "The train is late.", "I have a meeting at 10 AM.", 
    "Can you help me with this?", "The phone is ringing.", "We need to go now.", 
    "The sun is shining brightly.", "This is my favorite movie.", "He is very talented.", 
    "We will visit the museum tomorrow.", "The food tastes delicious.", "She is a good dancer.", 
    "He is waiting for the bus.", "What time is it?", "This is a difficult question.", 
    "The flowers are blooming.", "I need to finish my homework.", "Do you know how to cook?", 
    "The dog is barking loudly.", "She is reading a novel.", "The kids are playing outside.", 
    "What did you say?", "Please pass the salt.", "The store is closed.", "He is very kind.", 
    "This is the last one.", "I need a new phone.", "The movie starts at 7 PM.", 
    "She is wearing a red dress.", "I will call you later.", "We are planning a trip.", 
    "They are coming home soon.", "This is a beautiful painting.", "The concert was amazing.", 
    "I am learning French.", "The sun sets in the west.", "Can you hear me?", "It is very cold today.", 
    "He is writing a letter.", "The children are asleep.", "What is your favorite color?", 
    "She is a brilliant student.", "I forgot my keys.", "Do you have any questions?", 
    "The teacher is explaining the lesson.", "I will meet you at the park.", "Please be quiet.", 
    "The stars are shining in the sky.", "I have completed my project.", "The baby is crying.", 
    "She is watching a movie.", "I am making dinner.", "He is playing the guitar.", 
    "The train is arriving at the station.", "The house is very big.", "I am feeling better now.", 
    "The book is very interesting.", "She is going to the market.", "He is wearing a hat.", 
    "The birds are singing.", "The car is parked outside.", "I have lost my wallet.", 
    "She is baking a cake.", "He is studying for exams.", "The tree is very tall.", 
    "What is the problem?", "We need to buy groceries.", "He is painting the wall.", 
    "This is a serious issue.", "Can you open the door?", "The room is very clean.", 
    "I am going to the gym.", "The pizza is ready.", "He is a famous actor.", 
    "She is looking for her glasses.", "The hotel is fully booked.", "I will send you an email.", 
    "The water is boiling.", "We are visiting our grandparents.", "The road is under construction.", 
    "I am reading a magazine.", "The sky is clear.", "She is driving the car.", 
    "The dog is chasing the ball.", "The cake tastes sweet.", "He is playing a video game.", 
    "We are having a party tonight.", "The plane is taking off.", "It is very hot outside.", 
    "The movie was very entertaining.", "She is talking on the phone.", "I am learning how to swim."
]

reference_sentences = [
    "यह एक पेन है।", "आप कैसे हैं?", "किताब कहाँ है?", "वह स्कूल जा रही है।", 
    "बिल्ली छत पर है।", "वह एक डॉक्टर है।", "मुझे किताबें पढ़ना पसंद है।", "हम कोडिंग सीख रहे हैं।", 
    "वे फुटबॉल खेल रहे हैं।", "तुम्हारा नाम क्या है?", "आज मौसम अच्छा है।", 
    "तेज बारिश हो रही है।", "मैं बहुत थका हुआ हूँ।", "आपको स्वस्थ भोजन खाना चाहिए।", 
    "उसे गाना पसंद है।", "ट्रेन देर से है।", "मेरी 10 बजे बैठक है।", 
    "क्या आप इसमें मेरी मदद कर सकते हैं?", "फोन बज रहा है।", "हमें अब जाना चाहिए।", 
    "सूरज तेज़ चमक रहा है।", "यह मेरी पसंदीदा फिल्म है।", "वह बहुत प्रतिभाशाली है।", 
    "हम कल संग्रहालय जाएंगे।", "खाना बहुत स्वादिष्ट है।", "वह एक अच्छी नर्तकी है।", 
    "वह बस की प्रतीक्षा कर रहा है।", "समय क्या हो रहा है?", "यह एक कठिन सवाल है।", 
    "फूल खिल रहे हैं।", "मुझे अपना होमवर्क खत्म करना है।", "क्या आप खाना बनाना जानते हैं?", 
    "कुत्ता जोर से भौंक रहा है।", "वह एक उपन्यास पढ़ रही है।", "बच्चे बाहर खेल रहे हैं।", 
    "आपने क्या कहा?", "कृपया नमक पास करें।", "दुकान बंद है।", "वह बहुत दयालु है।", 
    "यह आखिरी है।", "मुझे एक नया फोन चाहिए।", "फिल्म 7 बजे शुरू होती है।", 
    "वह लाल पोशाक पहन रही है।", "मैं आपको बाद में कॉल करूंगा।", "हम एक यात्रा की योजना बना रहे हैं।", 
    "वे जल्द ही घर आ रहे हैं।", "यह एक सुंदर पेंटिंग है।", "संगीत कार्यक्रम अद्भुत था।", 
    "मैं फ्रेंच सीख रहा हूँ।", "सूरज पश्चिम में डूबता है।", "क्या आप मुझे सुन सकते हैं?", 
    "आज बहुत ठंड है।", "वह एक पत्र लिख रहा है।", "बच्चे सो रहे हैं।", "आपका पसंदीदा रंग कौन सा है?", 
    "वह एक प्रतिभाशाली छात्रा है।", "मैं अपनी चाबियाँ भूल गया।", "क्या आपके पास कोई सवाल है?", 
    "शिक्षक पाठ समझा रहा है।", "मैं पार्क में मिलूंगा।", "कृपया चुप रहें।", 
    "आसमान में सितारे चमक रहे हैं।", "मैंने अपना प्रोजेक्ट पूरा कर लिया है।", "बच्चा रो रहा है।", 
    "वह एक फिल्म देख रही है।", "मैं रात का खाना बना रहा हूँ।", "वह गिटार बजा रहा है।", 
    "ट्रेन स्टेशन पर पहुँच रही है।", "घर बहुत बड़ा है।", "अब मैं बेहतर महसूस कर रहा हूँ।", 
    "यह किताब बहुत दिलचस्प है।", "वह बाजार जा रही है।", "वह टोपी पहन रहा है।", 
    "पक्षी गा रहे हैं।", "गाड़ी बाहर खड़ी है।", "मेरा बटुआ खो गया है।", 
    "वह केक बना रही है।", "वह परीक्षाओं की तैयारी कर रहा है।", "पेड़ बहुत ऊँचा है।", 
    "समस्या क्या है?", "हमें किराने का सामान खरीदना है।", "वह दीवार पर रंग कर रहा है।", 
    "यह एक गंभीर मुद्दा है।", "क्या आप दरवाजा खोल सकते हैं?", "कमरा बहुत साफ है।", 
    "मैं जिम जा रहा हूँ।", "पिज्जा तैयार है।", "वह एक प्रसिद्ध अभिनेता है।", 
    "वह अपने चश्मे की तलाश कर रही है।", "होटल पूरी तरह से बुक है।", "मैं आपको एक ईमेल भेजूंगा।", 
    "पानी उबल रहा है।", "हम अपने दादा-दादी से मिलने जा रहे हैं।", "सड़क निर्माणाधीन है।", 
    "मैं एक पत्रिका पढ़ रहा हूँ।", "आसमान साफ है।", "वह कार चला रही है।", 
    "कुत्ता गेंद का पीछा कर रहा है।", "केक मीठा लगता है।", "वह वीडियो गेम खेल रहा है।", 
    "हम आज रात एक पार्टी कर रहे हैं।", "विमान उड़ान भर रहा है।", "बाहर बहुत गर्मी है।", 
    "फिल्म बहुत मनोरंजक थी।", "वह फोन पर बात कर रही है।", "मैं तैरना सीख रहा हूँ।"
]


# Tokenize and prepare the inputs
inputs = tokenizer(source_texts, return_tensors="pt", padding=True, truncation=True)

# Generate the translations
translated = model.generate(**inputs)
predicted_sentences = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Evaluate with sentence-level BLEU scores
bleu_scores = []
for ref, pred in zip(reference_sentences, predicted_sentences):
    ref_tokens = [ref.split()]
    pred_tokens = pred.split()
    bleu_score = sentence_bleu(ref_tokens, pred_tokens)
    bleu_scores.append(bleu_score)

# Calculate the average BLEU score
avg_bleu = np.mean(bleu_scores)

# Displaying the source, reference, prediction, and BLEU score for each sentence
for i, (source, ref, pred, bleu) in enumerate(zip(source_texts, reference_sentences, predicted_sentences, bleu_scores)):
    print(f"Source Text {i+1}: {source}")
    print(f"Reference Translation: {ref}")
    print(f"Predicted Translation: {pred}")
    print(f"BLEU Score: {bleu:.4f}\n")

print(f"Average BLEU Score: {avg_bleu:.4f}")


TypeError: Fraction.__new__() got an unexpected keyword argument '_normalize'